In [1]:
import pandas as pd
import numpy as np
import requests
import urllib
import re
from bs4 import BeautifulSoup

In [11]:
#Open the page with all albums.
main_page = 'https://www.azlyrics.com/a/atmosphere.html'
try:
    content = urllib.request.urlopen(main_page).read()
    main_soup = BeautifulSoup(content, 'html.parser')
except Exception as e:
    print('Error'+str(e))
#Pull the list of albums on the page and the associated IDs.
album_list = main_soup.find(id='listAlbum')
albums = [name.string.strip('\"') for name in album_list.findAll('b')]
album_ids = [number.get('id') for number in album_list.findAll(['a']) if len(number)==0]

In [104]:
#Create a dataframe containing each album and the ID
albumdf = pd.DataFrame(data={'AlbumName': albums, 'AlbumID': album_ids})
albumdf

,AlbumName,AlbumID
0,Overcast!,7391
1,Lucy Ford: The Atmosphere EPs,5001
2,God Loves Ugly,5005
3,Seven's Travels,3262
4,Headshots: SE7EN,7592
5,You Can't Imagine How Much Fun We're Having,5146
6,Happy Clown Bad Dub 8 / Fun EP,7187
7,Sad Clown Bad Summer #9,6298
8,Sad Clown Bad Fall #10,7048
9,Sad Clown Bad Winter #11,7050


In [161]:
#Need to implement loop/vectorize (if possible) so it can be done on every album.
album_dicts = dict.fromkeys(albumdf['AlbumID'])
for i,j in enumerate(albumdf['AlbumID']):
    #Find the album
    findalbum = album_list.findAll(class_='album')[i]
    #Initialize song list
    songs = []
    #Find where the song titles begin
    title = findalbum.findNext('a', text=True)
    #Loop through the list until reaching the next album, where there is an ID field.
    if i < len(albumdf)-1:
        while title.get('id') != albumdf['AlbumID'][i+1]:
            songs.append(title.text)
            title = title.findNext('a')
            print(title.text)
        album_dicts[j] = songs
    else:
        while title.text != 'Bar Food':
            songs.append(title.text)
            title = title.
        
        
        
        


Brief Description
Current Status
Complications
4:30 AM
Adjust
Clay
Sound Is Vibration
Multiples
Scapegoat
WND
Multiples (Reprise)
Caved In
Cuando Limpia El Humo
The Outernet
Primer

Like Today
Tears For The Sheep
Guns And Cigarettes
Don't Ever Fucking Question That
It Goes
If I Was Santa Claus
Aspiring Sociopath
Free Or Dead
Party For The Fight To Write
Mama Had A Baby And His Head Popped Off
They're All Gonna Laugh @ You
Lost And Found
The Woman With The Tattooed Hands
Nothing But Sunshine
Homecoming

The Bass And The Movement
Give Me
Fuck You Lucy
Hair
Godlovesugly
A Song About A Friend
Flesh
Saves The Day
Lovelife
Breathing
Vampires
A Girl Named Hope
Godlovesugly Reprise
Modern Man's Hustle
One Of A Kind
Blamegame
Shrapnel

Trying To Find A Balance
Bird Sings Why The Caged I Know
Reflections
Gotta Lotta Walls
The Keys To Life Vs. 15 Minutes Of Fame
Apple
Suicidegirls
Jason
Cats Van Bags
Los Angeles
Lift Her Pull Her
Shoes
National Disgrace
Denvemolorado
Liquor Lyles Cool July
Good T

In [160]:
album_list.findAll(class_='album')[18].text

'other songs:'

In [154]:
albumdf['AlbumID']

0      7391
1      5001
2      5005
3      3262
4      7592
5      5146
6      7187
7      6298
8      7048
9      7050
10     6710
11     6684
12     7049
13     7568
14    10118
15    15442
16    31840
17    43755
Name: AlbumID, dtype: object

In [159]:
album_list

<div id="listAlbum">
<a id="7391"></a><div class="album">album: <b>"Overcast!"</b> (1997)</div>
<a href="../lyrics/atmosphere/1597.html" target="_blank">1597</a><br/>
<a href="../lyrics/atmosphere/briefdescription.html" target="_blank">Brief Description</a><br/>
<a href="../lyrics/atmosphere/currentstatus.html" target="_blank">Current Status</a><br/>
<a href="../lyrics/atmosphere/complications.html" target="_blank">Complications</a><br/>
<a href="../lyrics/atmosphere/430am.html" target="_blank">4:30 AM</a><br/>
<a href="../lyrics/atmosphere/adjust.html" target="_blank">Adjust</a><br/>
<a href="../lyrics/atmosphere/clay.html" target="_blank">Clay</a><br/>
<a href="../lyrics/atmosphere/soundisvibration.html" target="_blank">Sound Is Vibration</a><br/>
<a href="../lyrics/atmosphere/multiples.html" target="_blank">Multiples</a><br/>
<a href="../lyrics/atmosphere/scapegoat.html" target="_blank">Scapegoat</a><br/>
<a href="../lyrics/atmosphere/wnd.html" target="_blank">WND</a><br/>
<a href="